In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import math
from datetime import datetime, timedelta, date
import time
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import matplotlib.patches as patches
from matplotlib.colors import TwoSlopeNorm
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from operator import itemgetter
from mapping_plot_attributes import *
from mapping_tickers import *
from utils import *
from download_data import DownloadData

In [2]:
tickers = list(magnificent_7_tickers.keys())
end_date = datetime.today()
hist_years, hist_months, hist_days = 1, 0, 0
start_date = datetime(end_date.year - hist_years, end_date.month - hist_months, end_date.day - hist_days)
tk_market = '^GSPC'

hist_data = DownloadData(end_date, start_date, tickers, tk_market)

downloaded_data = hist_data.download_yh_data(start_date, end_date, tickers, tk_market)
df_adj_close = downloaded_data['Adj Close']
df_close = downloaded_data['Close']
df_volume = downloaded_data['Volume']
dict_ohlc = downloaded_data['OHLC']

tk = 'AAPL'
df_ohlc = dict_ohlc[tk]
ohlc_tk = df_ohlc.copy()
adj_close_tk = df_adj_close[tk]
close_tk = df_close[tk]
open_tk = ohlc_tk['Open']
high_tk = ohlc_tk['High']
low_tk = ohlc_tk['Low']
volume_tk = df_volume[tk]

price_type_map = {
    'Adj Close': adj_close_tk,
    'Adjusted Close': adj_close_tk,
    'Close': close_tk,
    'Open': open_tk,
    'High': high_tk,
    'Low': low_tk
}

# display(df_adj_close)
# display(df_close)
# display(df_ohlc)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

The portfolio data will be truncated to end at the latest available date of 2024-10-02.


In [3]:
def weighted_mean(values):
    """
    values: a list, tuple or series of numerical values
    """
    if isinstance(values, (list, tuple)):
        values = pd.Series(values)
    
    n = len(values)
    weight_sum = n * (n + 1) / 2
    weights = range(n + 1)[1:]
    wm = values @ weights / weight_sum
    return wm

In [4]:
def wilder_moving_average(df_tk, n):
    """
     J. Welles Wilder's EMA 
     https://stackoverflow.com/questions/40256338/calculating-average-true-range-atr-on-ohlc-data-with-python
    """
    wwma = df_tk.ewm(alpha = 1 / n, adjust = False).mean()
    return wwma

In [5]:
def average_true_rate(
    close_tk,
    high_tk,
    low_tk,
    n = 14
):
    """
    https://stackoverflow.com/questions/40256338/calculating-average-true-range-atr-on-ohlc-data-with-python
    
    """
    tr_cols = ['tr0', 'tr1', 'tr2']
    df_tr = pd.DataFrame(columns = tr_cols, index = close_tk.index)

    df_tr['tr0'] = abs(high_tk - low_tk)
    df_tr['tr1'] = abs(high_tk - close_tk.shift())
    df_tr['tr2'] = abs(low_tk - close_tk.shift())
    tr = df_tr[tr_cols].max(axis = 1)

    atr = wilder_moving_average(tr, n)
    atrp = atr / close_tk * 100
    atr_data = {
        'atr': atr,
        'atrp': atrp
    }
    
    return atr_data

In [6]:
atr_data = average_true_rate(close_tk, high_tk, low_tk)
atr = atr_data['atr']
atrp = atr_data['atrp']
display(atr)
print(atr.max())

Date
2023-10-03    2.809998
2023-10-04    2.840712
2023-10-05    2.835662
2023-10-06    2.976687
2023-10-09    2.996209
                ...   
2024-09-26    4.726946
2024-09-27    4.547878
2024-09-30    4.595173
2024-10-01    4.928375
2024-10-02    4.887062
Length: 252, dtype: float64

6.920488659270815


In [7]:
def moving_average(
    df_tk,
    ma_type,
    ma_window,
    min_periods = 1
):
    """
    df_tk:      
        a series of price values, taken as a column of df_close or df_adj_close for ticker tk
    ma_type:    
        simple ('sma'),
        exponential ('ema'),
        double exponential ('dema'),
        triple exponential ('tema'),
        weighted ('wma')
    window:
        length in days
    Returns ma
    """

    if not isinstance(df_tk, pd.Series):
        print('Incorrect format of input data')
        exit
    
    if ma_type in ['ema', 'dema', 'tema']:
        ma = df_tk.ewm(span = ma_window).mean()
        if ma_type in ['dema', 'tema']:
            ma = ma.ewm(span = ma_window).mean()
            if ma_type == 'tema':
                ma = ma.ewm(span = ma_window).mean()
    
    elif ma_type == 'wma':
        ma = df_tk.rolling(window = ma_window, min_periods = min_periods).apply(lambda x: weighted_mean(x))

    else:  # 'sma' or anything else
        ma = df_tk.rolling(window = ma_window, min_periods = min_periods).mean()
    
    return ma

In [8]:
wma = moving_average(close_tk, 'wma', 14)
wwma = wilder_moving_average(close_tk, 14)
display(wma - wwma)

Date
2023-10-03    0.000000
2023-10-04    0.750006
2023-10-05    1.412150
2023-10-06    2.433354
2023-10-09    3.189044
                ...   
2024-09-26    1.097252
2024-09-27    1.390563
2024-09-30    1.961768
2024-10-01    2.035860
2024-10-02    2.119482
Name: AAPL, Length: 252, dtype: float64

In [9]:
ma_list = [
    {
        'ma_idx': 1,
        'ma_type': 'sma',
        # 'ma_window': 10,
        'ma_window': 5,
        'showlegend': True
    },
    {
        'ma_idx': 2,
        'ma_type': 'sma',
        # 'ma_window': 20,
        'ma_window': 10,
        'showlegend': True
    },
    {
        'ma_idx': 3,
        'ma_type': 'sma',
        # 'ma_window': 30,
        'ma_window': 15,
        'showlegend': True
    },
    {
        'ma_idx': 4,
        'ma_type': 'sma',
        # 'ma_window': 40,
        'ma_window': 20,
        'showlegend': True
    }
    ,
    {
        'ma_idx': 5,
        'ma_type': 'sma',
        #'ma_window': 50,
        'ma_window': 25,
        'showlegend': True
    },
    {
        'ma_idx': 6,
        'ma_type': 'sma',
        # 'ma_window': 60,
        'ma_window': 30,
        # 'showlegend': True
        'showlegend': True
    }
]

ema_list = [
    {
        'ma_idx': 1,
        'ma_type': 'ema',
        'ma_window': 10,
        'showlegend': True
    },
    {
        'ma_idx': 2,
        'ma_type': 'ema',
        'ma_window': 20,
        'showlegend': True
    },
    {
        'ma_idx': 3,
        'ma_type': 'ema',
        'ma_window': 30,
        'showlegend': True
    },
    {
        'ma_idx': 4,
        'ma_type': 'ema',
        'ma_window': 40,
        'showlegend': True
    }
    ,
    {
        'ma_idx': 5,
        'ma_type': 'ema',
        'ma_window': 50,
        'showlegend': True
    },
    {
        'ma_idx': 6,
        'ma_type': 'ema',
        'ma_window': 60,
        'showlegend': True
    }
]

In [10]:
price_list = [
    {
        'name': 'Adjusted Close',
        'data': adj_close_tk,
        'show': False
    },
    {
        'name': 'Open',
        'data': ohlc_tk['Open'],
        'show': True
    },
    {
        'name': 'Close',
        'data': close_tk,
        'show': True
    },
    {
        'name': 'Low',
        'data': ohlc_tk['Low'],
        'show': False
    },
    {
        'name': 'High',
        'data': ohlc_tk['High'],
        'show': False
    }
]

In [11]:
##### NEEDS FIXING #####

def update_color_theme(
    fig_data,
    theme,
    color_theme,
    target_deck = None,
    invert = False
):
    """
    fig = fig_data['fig']
    theme: existing theme ('dark' or light')
    color_theme: new color theme to apply to overlays in fig
    target_deck: equivalent to legendgroup - 1, 2, or 3 or all of them if None
    invert: invert the palette from lightest-darkest to darkest-lightest or vice versa

    Returns updated fig
    """
    # n_decks = max([tr['legendgroup'] for tr in fig_data['fig']['data'] if tr['legendgroup'].isdigit()])

    style = theme_style[theme]
    overlay_colors = style['overlay_color_theme'][color_theme]
    color_map = fig_data['color_map']

    for name, color_idx in color_map.items():

        if target_deck is not None:
            # update for a single deck only
            selector = dict(name = name, legendgroup = str(target_deck))
        else:
            # update for all decks
            selector = dict(name = name)

        if invert:
            color_idx = len(color_map) - color_idx - 1

        fig_data['fig'].update_traces(
            line_color = overlay_colors[color_idx],
            selector = selector
        )
    
    return fig_data

In [403]:
def adjust_legend_position(
    fig_data,
    deck_type,
    legend_title_height = 21
):
    """  99
    legend_title_height:
        upper title height to be subtracted for triple deck, depends on the legend title font size, 21 is for size 16
    """
    
    n_traces_upper = len([x for x in fig_data['fig']['data'] if (x['legendgroup'] == '1') & (x['showlegend'] if x['showlegend'] is not None else True)])
    n_traces_middle = len([x for x in fig_data['fig']['data'] if (x['legendgroup'] == '2') & (x['showlegend'] if x['showlegend'] is not None else True)])
    n_traces_lower = len([x for x in fig_data['fig']['data'] if (x['legendgroup'] == '3') & (x['showlegend'] if x['showlegend'] is not None else True)])
    n_traces_total = n_traces_upper + n_traces_middle + n_traces_lower

    print('n_traces_upper, n_traces_middle, n_traces_lower')
    print(n_traces_upper, n_traces_middle, n_traces_lower)

    legend_item_height = abs(legend_gap['double']['slope'])

    # NOTE: The middle and lower plots in the triple deck should be of the same height
    height_upper = fig_data['plot_height'][1]
    height_lower = fig_data['plot_height'][2]

    a0 = -118.3
    a_upper = 0.94258
    a_lower = 0.17954
  
    if (deck_type == 'double') | (n_traces_lower == 0):
        
        # a0 = legend_gap['double']['intercept'][height_upper][height_lower]
        
        intercept = a0 + a_upper * height_upper + a_lower * height_lower
        legend_groupgap = intercept - legend_item_height * n_traces_upper

    elif deck_type == 'triple':

        if n_traces_middle == 0:
            
            # a0 = legend_gap['double']['intercept'][(height_upper + height_lower)][height_lower]
            # legend_groupgap = a0 - legend_item_height * n_traces_upper - legend_title_height

            intercept = a0 + a_upper * (height_upper + height_lower) + a_lower * height_lower
            legend_groupgap = intercept - n_traces_upper * legend_item_height - legend_title_height
            
        else:
            # a0 = legend_gap['triple']['intercept'][height_lower]
            # a1 = legend_gap['triple']['slope'][height_lower]
            
            a0 = -177
            legend_groupgap = (a0 + height_upper + 2 * height_lower - n_traces_total * legend_item_height - 3 * legend_title_height) / 2
 
    legend_tracegroupgap = max(legend_groupgap, 0)
    print(legend_tracegroupgap)

    return legend_tracegroupgap

In [91]:
def add_overlay(
    fig_data,
    df,
    name,
    color_idx,
    showlegend = True,
    target_deck = 1,
    theme = 'dark',
    color_theme = 'gold'
):
    """
    fig_data: a dictionary of the underlying figure data

    y_min_fig: y_min on the existing fig
    y_max_fig: y_max on the existing fig
    color_idx: an integer (0, ...) indicating the color from those available in theme_style
    showlegend: whether or not to show line in legend (e.g. we only need one Bollinger band in legend)
    legendgroup: 'upper' is top graph in subplots (row 1), 'lower' is the stacked lower graph (row 2)

    Returns the updated fig_data dictionary
    """

    style = theme_style[theme]
    overlay_colors = style['overlay_color_theme'][color_theme]

    fig = fig_data['fig']
    y_min_fig = fig_data['y_min'][target_deck]
    y_max_fig = fig_data['y_max'][target_deck]
    deck_type = fig_data['deck_type']

    min_y = min(df)
    max_y = max(df)
    y_min, y_max = set_axis_limits(min_y, max_y)

    # ESSENTIALLY THERE SHOULD BE NO OVERLAYS ADDED TO AN EMPTY DECK so this may not be necessary
    try:
        new_y_min, new_y_max = set_axis_limits(min(y_min, y_min_fig), max(y_max, y_max_fig))
    except:
        # if the existing y_min and y_max are None
        new_y_min, new_y_max = y_min, y_max

    # print(f'{name}\n\tmin_y = {min_y}, max_y = {max_y}')
    # print(f'set_axis_limits: {set_axis_limits(min_y, max_y)}')
    # print(f'{name}\n\ty_min = {y_min}, y_max = {y_max}\n\tnew_y_min = {new_y_min}, new_y_max = {new_y_max}')

    if target_deck > 1:
        new_y_max *= 0.999

    if color_idx >= len(overlay_colors):
        # Take the last overlay color from the available list
        color_idx = -1

    legendgrouptitle = {}
    if deck_type == 'triple':
        legendtitle = tripledeck_legendtitle[target_deck]
        legendgrouptitle = dict(
            text = legendtitle,
            font_size = 16,
            font_weight = 'bold'
        )

    fig.add_trace(
        go.Scatter(
            x = df.index.astype(str),
            y = df,
            line = dict(color = overlay_colors[color_idx]),
            name = name,
            showlegend = showlegend,
            legendgroup = f'{target_deck}',
            legendgrouptitle = legendgrouptitle
        ),
        row = target_deck, col = 1    
    )

    fig.update_yaxes(
        range = (new_y_min, new_y_max),
        showticklabels = True,
        ticks = 'outside',
        ticklen = 8,
        row = target_deck, col = 1
    )

    fig_data.update({'fig': fig})
    fig_data['y_min'].update({target_deck: new_y_min})
    fig_data['y_max'].update({target_deck: new_y_max})

    return fig_data

In [67]:
def add_ma_overlays(
    fig_data,
    df_price,
    ma_list,
    target_deck = 1,
    x_min = None,
    x_max = None,
    add_yaxis_title = False,
    yaxis_title = 'Moving Average',
    theme = 'dark',
    color_theme = 'gold'
):
    """
    df_price: 
        df_close or df_adj_close, depending on the underlying figure
    ma_list: 
        list of ma overlay dictionaries, containing
         - ma_idx ma index (1, 2,...)
         - ma_type: 'sma' (default), 'ema', 'dema', 'tema', 'wma' or 'wwma'
         - ma_window, in days
         - showlegend: include in plot legend or not
    """

    x_min = start_date if x_min is None else x_min
    x_max = end_date if x_max is None else x_max

    deck_type = fig_data['deck_type']

    n_ma = len(ma_list)

    style = theme_style[theme]
    overlay_color_idx = style['overlay_color_selection'][color_theme][n_ma]

    current_names = [trace['name'] for trace in fig_data['fig']['data'] if (trace['legendgroup'] == str(target_deck))]
    
    ma_overlays = []

    for i, ma in enumerate(ma_list):
        
        ma_type = ma['ma_type']
        ma_window = ma['ma_window']
        ma_name = f'{ma_type.upper()} {ma_window}'

        if ma_name not in current_names:  # MUST ALSO CHECK THE DECK
    
            ma_data = moving_average(
                df_price[x_min: x_max],
                ma_type,
                ma_window
            )
            ma_color_idx = overlay_color_idx[i]
            ma_showlegend = ma['showlegend']

            ma_overlays.append({
                'data': ma_data,
                'name': ma_name,
                'color_idx': ma_color_idx,
                'showlegend': ma_showlegend
            })

    # color_map = fig_data['color_map']
    color_map = {}

    for overlay in ma_overlays:
        fig_data = add_overlay(
            fig_data,
            overlay['data'],
            overlay['name'],
            overlay['color_idx'],
            overlay['showlegend'],
            target_deck = target_deck,
            theme = theme,
            color_theme = color_theme
        )        
        color_map.update({overlay['name']: overlay['color_idx']})

    if deck_type in ['double', 'triple']:
        legend_tracegroupgap = adjust_legend_position(fig_data, deck_type)
        fig_data['fig'].update_layout(
            legend_tracegroupgap = legend_tracegroupgap,
            legend_traceorder = 'grouped'
        )
    
    if add_yaxis_title:
        fig_data['fig'].update_yaxes(
            title = yaxis_title,
            row = target_deck, col = 1
        )

    fig_data.update({'color_map': color_map})

    return fig_data

In [68]:
def add_price_overlays(
    fig_data,
    price_list,
    x_min = None,
    x_max = None,
    target_deck = 1,
    add_yaxis_title = False,
    yaxis_title = 'Price',
    theme = 'dark',
    color_theme = 'gold'
):
    """
    fig_data:
        A dictionary containing the underlying figure data
    price_list: 
        list of dictionaries with keys
         - 'name': 'Adjusted Close', 'Close', 'Open', 'High', 'Low', 'Average True Rate', etc.
         - 'show': True / False - include in plot or not
    x_min, x_max:
        minimum and maximum dates in the datetime format
    """
    
    x_min = start_date if x_min is None else x_min
    x_max = end_date if x_max is None else x_max

    deck_type = fig_data['deck_type']

    # Count lines that will be overlaid ('show' is True)

    selected_prices = [x for x in price_list if x['show']]
    n_price = len(selected_prices)

    style = theme_style[theme]
    overlay_color_idx = style['overlay_color_selection'][color_theme][n_price]

    current_names = [trace['name'] for trace in fig_data['fig']['data'] if (trace['legendgroup'] == str(target_deck))]
    
    price_overlays = []

    for i, price in enumerate(selected_prices):
        
        price_name = price['name']
    
        if price_name not in current_names:

            price_data = price['data']  # [x_min: x_max]
            color_idx = overlay_color_idx[i]

            price_overlays.append({
                'data': price_data,
                'name': price_name,
                'color_idx': color_idx
            })

    color_map = {}

    for overlay in price_overlays:
        fig_data = add_overlay(
            fig_data,
            overlay['data'],
            overlay['name'],
            overlay['color_idx'],
            target_deck = target_deck,
            theme = theme,
            color_theme = color_theme
        )        
        color_map.update({overlay['name']: overlay['color_idx']})

    if deck_type in ['double', 'triple']:
        legend_tracegroupgap = adjust_legend_position(fig_data, deck_type)
        fig_data['fig'].update_layout(
            legend_tracegroupgap = legend_tracegroupgap,
            legend_traceorder = 'grouped'
        )

    if add_yaxis_title:
        fig_data['fig'].update_yaxes(
            title = yaxis_title,
            row = target_deck, col = 1
        )
        
    fig_data.update({'color_map': color_map})

    return fig_data

In [99]:
def get_macd(
    df_tk,
    signal_window = 9      
):
    """
    df_tk: a series of price values, taken as a column of df_close or df_adj_close for ticker tk
    """ 

    if not isinstance(df_tk, pd.Series):
        print('Incorrect format of input data')
        exit
    
    ema_26 = df_tk.ewm(span = 26).mean()
    ema_12 = df_tk.ewm(span = 12).mean()
    macd_line = ema_12 - ema_26
        
    macd_signal = macd_line.ewm(span = signal_window).mean()
    macd_histogram = macd_line - macd_signal

    macd_data = {
        'MACD': macd_line,
        'MACD Signal': macd_signal,
        'MACD Signal Window': signal_window,
        'MACD Histogram': macd_histogram
    }

    return macd_data

In [98]:
def get_macd_v(
    close_tk,
    high_tk,
    low_tk,
    signal_window = 9      
):
    """
    close_tk, high_tk, low_tk: 
        series of Close, High and Low daily price values for ticker tk
    atr_data:
        output from average_true_rate(), containing the ATR and ATRP lines
    """ 

    if not isinstance(close_tk, pd.Series):
        print('Incorrect format of input data')
        exit
    
    atr_data = average_true_rate(close_tk, high_tk, low_tk, n = 26)
    atr = atr_data['atr']

    ema_26 = close_tk.ewm(span = 26).mean()
    ema_12 = close_tk.ewm(span = 12).mean()

    macd_v_line = 100 * (ema_12 - ema_26) / atr
        
    macd_v_signal = macd_v_line.ewm(span = signal_window).mean()
    macd_v_histogram = macd_v_line - macd_v_signal

    macd_v_data = {
        'MACD': macd_v_line,
        'MACD Signal': macd_v_signal,
        'MACD Signal Window': signal_window,
        'MACD Histogram': macd_v_histogram
    }

    return macd_v_data

In [286]:
def add_macd(
    fig_data,
    tk_macd,
    macd_data,
    volatility_normalized = True,
    histogram_type = 'macd-signal',
    include_signal = True,
    n_yticks_max = 7,
    target_deck = 2,
    add_title = False,
    title_font_size = 32,
    theme = 'dark'
):
    """
    Adds MACD with a signal line to a stacked plot
    volatility_normalized:
        treat input macd_data as MACD-V
    histogram_type:
        'macd-signal': MACD and Signal will be plotted as lines, 
            the green-red histogram will be based on their difference
        'macd-zero': MACD will be plotted as the green-red histogram,
            and the Signal as line will be based on their difference
    include_signal:
        this will plot signal and MACD/MACD-V lines in addition to the histogram
    """
    
    # x_min = start_date if x_min is None else x_min
    # x_max = end_date if x_max is None else x_max

    fig_macd = fig_data['fig']
    deck_type = fig_data['deck_type']

    style = theme_style[theme]

    if volatility_normalized:
        yaxis_title = f'MACD-V'
    else:
        yaxis_title = f'MACD'

    macd = macd_data['MACD']
    macd_histogram = macd_data['MACD Histogram']
    
    if histogram_type == 'macd-signal':
        macd_legend_positive = f'{yaxis_title} > Signal'
        macd_legend_negative = f'{yaxis_title} < Signal'
    else:
        macd_legend_positive = f'{yaxis_title} > 0'
        macd_legend_negative = f'{yaxis_title} < 0'

    if include_signal:
        macd_signal = macd_data['MACD Signal']
        macd_signal_window = macd_data['MACD Signal Window']
        min_macd = min(min(macd), min(macd_signal))
        max_macd = max(max(macd), max(macd_signal))
    else:
        if histogram_type == 'macd-signal':
            min_macd = min(macd_histogram)
            max_macd = max(macd_histogram)
        else:
            min_macd = min(macd)
            max_macd = max(macd)

    y_macd_min, y_macd_max = set_axis_limits(min_macd, max_macd, max_n_intervals = 8)
    if target_deck > 1:
        y_macd_max *= 0.999

    legendgrouptitle = {}
    if deck_type == 'triple':
        legendtitle = tripledeck_legendtitle[target_deck]
        legendgrouptitle = dict(
            text = legendtitle,
            font_size = 16,
            font_weight = 'bold'
        )

    #####

    if histogram_type == 'macd-zero':

        macd_positive = macd.copy()
        macd_positive.iloc[np.where(macd_positive < 0)] = np.nan
        macd_negative = macd.copy()
        macd_negative.iloc[np.where(macd_negative >= 0)] = np.nan
    
        fig_macd.add_trace(
            go.Bar(
                x = macd_positive.index.astype(str),
                y = macd_positive,
                marker_color = style['green_color'],
                width = 1,
                name = macd_legend_positive,
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                showlegend = True
            ),
            row = target_deck, col = 1
        )
        fig_macd.add_trace(
            go.Bar(
                x = macd_negative.index.astype(str),
                y = macd_negative,
                marker_color = style['red_color'],
                width = 1,
                name = macd_legend_negative,
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                showlegend = True
            ),
            row = target_deck, col = 1
        )

    else:
        # histogram_type is 'macd-signal' (default)

        macd_histogram_positive = macd_histogram.copy()
        macd_histogram_positive.iloc[np.where(macd_histogram_positive < 0)] = np.nan
        macd_histogram_negative = macd_histogram.copy()
        macd_histogram_negative.iloc[np.where(macd_histogram_negative >= 0)] = np.nan
    
        fig_macd.add_trace(
            go.Bar(
                x = macd_histogram_positive.index.astype(str),
                y = macd_histogram_positive,
                marker_color = style['green_color'],
                width = 1,
                name = macd_legend_positive,
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                showlegend = True
            ),
            row = target_deck, col = 1
        )
        fig_macd.add_trace(
            go.Bar(
                x = macd_histogram_negative.index.astype(str),
                y = macd_histogram_negative,
                marker_color = style['red_color'],
                width = 1,
                name = macd_legend_negative,
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                showlegend = True
            ),
            row = target_deck, col = 1
        )

    if include_signal:
        fig_macd.add_trace(
            go.Scatter(
                x = macd.index.astype(str),
                y = macd,
                line = dict(color = style['basecolor']),
                # name = 'Signal Line'  # 9-day span is a standard, no need to customise it
                name = yaxis_title,
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                showlegend = True
            ),
            row = target_deck, col = 1
        )
        fig_macd.add_trace(
            go.Scatter(
                x = macd_signal.index.astype(str),
                y = macd_signal,
                line = dict(color = style['signal_color']),
                # name = 'Signal Line'  # 9-day span is a standard, no need to customise it
                name = f'EMA {macd_signal_window} Signal',
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                showlegend = True
            ),
            row = target_deck, col = 1
        )


    if deck_type in ['double', 'triple']:
        legend_tracegroupgap = adjust_legend_position(fig_data, deck_type)
        fig_data['fig'].update_layout(
            legend_tracegroupgap = legend_tracegroupgap,
            legend_traceorder = 'grouped'
        )

    if add_title & (target_deck == 1):
        
        if volatility_normalized:
            title_macd = f'{tk_macd} Volatility-Normalized MACD(12, 26)'
        else:
            title_macd = f'{tk_macd} MACD(12, 26)'

        fig_macd.update_layout(
            title = dict(
                text = title_macd,
                font_size = title_font_size,
                y = 0.98,
                x = 0.45,
                xanchor = 'center',
                yanchor = 'top'
            )
        )

    fig_macd.update_yaxes(
        title_text = yaxis_title,
        range = (y_macd_min, y_macd_max),
        showticklabels = True,        
        nticks = n_yticks_max,
        row = target_deck, col = 1
    )

    fig_data.update({'fig': fig_macd})
    fig_data['y_min'].update({target_deck: y_macd_min})
    fig_data['y_max'].update({target_deck: y_macd_max})

    return fig_data 

There should be a limit, preferably 1000 px, on the total height of a stacked graph. 

For double stacks could ask user for upper and lower heights, with a 250 limit on the lower height if the upper height is 750.<BR>
The upper height should be 300, 450, 600 or 750.<BR>
The lower height should be 100, 150, 200, 250 or 300 for all upper heights except 750.<BR>
The idea is that the upper height is always larger than the lower height in a double deck and larger than or equal to the lower height in a triple deck.<BR>

A drop-down list of upper/lower height combinations may be more efficient than asking for two heights separately.<BR>
Or maybe a double drop-down list where for each upper height selected from the primary list there will be a list of lower heights available from the secondary list, if that's possible.

UPPER / LOWER<BR>
750 / 250<BR>
750 / 200<BR>
750 / 150<BR>
750 / 100<BR>
600 / 300<BR>
600 / 250<BR>
600 / 200<BR>
600 / 150<BR>
600 / 100<BR>
450 / 300<BR>
450 / 250<BR>
450 / 200<BR>
450 / 150<BR>
450 / 100<BR>

The middle and lower plot heights should preferably be equal for aesthetic reasons.

Example combinations of heights for a triple-stacked graph:

Upper &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Middle &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Lower<BR>
750 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 100 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 100<BR>
600 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 100-200 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 100-200<BR>
450 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 100-200 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 100-200<BR>
300 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 100-300 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 100-300<BR>

For triple stacks, it might be best to offer pre-specified height combos, for example:

UPPER / MIDDLE / LOWER<BR>
750 / 100 / 100<BR>
600 / 200 / 200<BR>
600 / 150 / 150<BR>
450 / 200 / 200<BR>
450 / 150 / 150<BR>
300 / 300 / 300<BR>
300 / 250 / 250<BR>
300 / 200 / 200<BR>
300 / 150 / 150<BR>

In [18]:
def create_template(
    date_index,
    deck_type = 'triple',
    plot_width = 1600,
    n_ticks_max = 52,
    plot_height_1 = None,
    plot_height_2 = None,
    plot_height_3 = None,
    n_yticks_max_1 = 12,
    n_yticks_max_2 = 7,
    n_yticks_max_3 = 7,
    theme = 'dark'
):
    """
    Info whether the deck is a single, double or triple will come from user's input.
    Then the name of the deck (deck_type) will be translated into a number;, e.g. 'lower'
    will be translated to 2 in a double deck, while 'middle' and 'lower' will be translated 
    to 2 and 3, respectively, in a triple deck.
    
    legendgrouptitle will be an empty dictionary for a single and double deck, and will
    be populated with the appropriate deck name in a triple deck.

    date_index:
        series or list of dates (e.g. close_tk.index)
    deck_type:
        'single', 'double' or 'triple'
    
    The default n_yticks values should really be some function of plot height,
    except in special cases where it can be specified customly.
    
    There should be a separate function to update the y axis in any deck based
    on the custom-specified number of ticks.
    Likewise to update the x axis to select a different width (1280, 1450, 1600)
    Defaults could be: n_yticks_max = {1: 12, 2: 7, 3: 7}
    
    """

    map_deck_type = {'single': 1, 'double': 2, 'triple': 3}
    n_rows = map_deck_type[deck_type]

    # Set up dictionaries for convenience

    plot_height = {}

    if (deck_type == 'single'):
        plot_height_1 = 750 if plot_height_1 is None else plot_height_1
        plot_height.update({1: plot_height_1})
    
    elif (deck_type == 'double'):
        plot_height_1 = 750 if plot_height_1 is None else plot_height_1
        plot_height_2 = 250 if plot_height_2 is None else plot_height_2
        plot_height.update({
            1: plot_height_1,
            2: plot_height_2
        })
    
    elif (deck_type == 'triple'):
        plot_height_1 = 600 if plot_height_1 is None else plot_height_1
        plot_height_2 = 200 if plot_height_2 is None else plot_height_2
        plot_height_3 = 200 if plot_height_3 is None else plot_height_3
        plot_height.update({
            1: plot_height_1,
            2: plot_height_2,
            3: plot_height_3
        })
    
    n_yticks = {
        1: n_yticks_max_1,
        2: n_yticks_max_2,
        3: n_yticks_max_3
    }
    
    df_dummy = pd.Series(index = date_index)
    for _, idx in enumerate(date_index):
        df_dummy[idx] = 0

    x_min = str(min(df_dummy.index).date())
    x_max = str(max(df_dummy.index).date())

    height_pct = {}
    row_heights = []
    plot_height_total = sum(h for h in plot_height.values())
    for k, h in plot_height.items():
        h_pct = h / plot_height_total
        height_pct.update({k: h_pct})
        row_heights.append(h_pct)

    if deck_type == 'single':
        y_range = {
            1: {
                'y0': 0,
                'y1': 1
            }
        }
    elif deck_type == 'double': 
        y_range = {
            1: {
                'y0': height_pct[2],
                'y1': 1
            },
            2: {
                'y0': 0,
                'y1': height_pct[2]
            }
        }
    elif deck_type == 'triple': 
        y_range = {
            1: {
                'y0': height_pct[2] + height_pct[3],
                'y1': 1
            },
            2: {
                'y0': height_pct[3],
                'y1': height_pct[2] + height_pct[3]
            },
            3: {
                'y0': 0,
                'y1': height_pct[3]
            }
        }

    style = theme_style[theme]
    
    fig = make_subplots(
        rows = n_rows,
        cols = 1,
        shared_xaxes = True,
        vertical_spacing = 0,
        row_heights = row_heights
    )

    for k in range(n_rows + 1)[1:]:

        # Add dummy traces
        fig.add_trace(
            go.Scatter(
                x = df_dummy.index.astype(str),
                y = df_dummy,
                line_width = 0,         
                showlegend = False,     
                legendgroup = 'dummy',
                hoverinfo = 'skip'
            ),
            row = k, col = 1
        )

        # Add plot borders
        fig.add_shape(
            type = 'rect',
            xref = 'x',  # use 'x' because 'paper' does not work correctly with stacked plots
            yref = 'paper',
            x0 = x_min,
            x1 = x_max,
            y0 = y_range[k]['y0'],
            y1 = y_range[k]['y1'],
            line_color = style['x_linecolor'],
            line_width = 2
        )

        # Update axes
        fig.update_xaxes(
            type = 'category',
            showgrid = True,
            gridcolor = style['x_gridcolor'],
            nticks = n_ticks_max,
            tickangle = -90,
            ticks = 'outside',
            ticklen = 8,
            row = k, col = 1
        )
        fig.update_yaxes(
            showgrid = True,
            gridcolor = style['y_gridcolor'],
            zerolinecolor = style['x_gridcolor'],
            zerolinewidth = 1,
            nticks = n_yticks[k],
            ticks = 'outside',
            ticklen = 8,
            showticklabels = False,
            row = k, col = 1
        )

    # Update layout
    fig.update_layout(
        margin_t = 60,
        width = plot_width,
        height = plot_height_total,
        xaxis_rangeslider_visible = False,
        template = style['template'],
        legend_groupclick = 'toggleitem'
    )

    y_min = {1: None, 2: None, 3: None}
    y_max = {1: None, 2: None, 3: None}
    
    fig_data = {
        'fig': fig,
        'y_min': y_min,
        'y_max': y_max,
        'plot_height': plot_height,
        'deck_type': deck_type,
        'color_map': {}
    }

    return fig_data

In [19]:
tripledeck_legendtitle = {
    1: 'UPPER',
    2: 'MIDDLE',
    3: 'LOWER'
} 

In [108]:
def add_candlestick(
    fig_data,
    df_ohlc,
    tk,
    candle_type = 'hollow',
    target_deck = 1,
    add_title = True,
    title_font_size = 32,
    theme = 'dark'
):
    """
    candle_type: 'hollow' or 'traditional'
        
    """
    
    style = theme_style[theme]

    # Colors must be in the RGBA format
    red_color = style['red_color']
    green_color = style['green_color']
    red_fill_color = red_color
    green_fill_color = green_color
    red_fill_color_hollow = red_color.replace(', 1)', ', 0.2)')
    green_fill_color_hollow = green_color.replace(', 1)', ', 0.2)')

    df = df_ohlc.copy()

    min_y = min(df['Low'])
    max_y = max(df['High'])
    y_min, y_max = set_axis_limits(min_y, max_y)

    if target_deck > 1:
        y_max *= 0.999

    df['Date'] = df.index.astype(str)

    fig = fig_data['fig']
    deck_type = fig_data['deck_type']

    legendgrouptitle = {}
    if deck_type == 'triple':
        legendtitle = tripledeck_legendtitle[target_deck]
        legendgrouptitle = dict(
            text = legendtitle,
            font_size = 16,
            font_weight = 'bold'
        )

    if candle_type == 'traditional':

        if add_title:
            title = f'{tk} Prices - Traditional Candles'

        shown_green = False
        shown_red = False

        for _, row in df.iterrows():

            if row['Close'] >= row['Open']:
                color_dict = dict(
                    fillcolor = green_fill_color,
                    line = dict(color = green_color)
                )
                name = 'Close > Open'
                current_candle = 'green'
            else:
                color_dict = dict(
                    fillcolor = red_fill_color,
                    line = dict(color = red_color)
                )
                name = 'Open > Close'
                current_candle = 'red'

            # Make sure each candle type appears only once in the legend
            if (not shown_green) & (current_candle == 'green'):
                showlegend = True
                shown_green = True
            elif (not shown_red) & (current_candle == 'red'):
                showlegend = True
                shown_red = True
            else:
                showlegend = False

            fig.add_trace(
                go.Candlestick(
                    x = [row['Date']],
                    open = [row['Open']],
                    high = [row['High']],
                    low = [row['Low']],
                    close = [row['Close']],
                    name = name,
                    increasing = color_dict,
                    decreasing = color_dict,
                    showlegend = showlegend,
                    legendgroup = f'{target_deck}',
                    legendgrouptitle = legendgrouptitle
                ),
                row = target_deck, col = 1
            )
        
    else:  # candle_type == 'hollow'
        
        if add_title:
            title = f'{tk} Prices - Hollow Candles'
        
        df['previousClose'] = df['Close'].shift(1)
        
        # Define color based on close and previous close
        df['color'] = np.where(df['Close'] > df['previousClose'], green_color, red_color)
        
        df['fill'] = np.where(
            df['color'] == green_color,
            np.where(df['Close'] > df['Open'], green_fill_color_hollow, green_color),
            np.where(df['Close'] > df['Open'], red_fill_color_hollow, red_color)
        )
        
        shown_red_fill = False
        shown_red_hollow = False
        shown_green_fill = False
        shown_green_hollow = False
        
        for _, row in df.iterrows():
            
            if (row['color'] == green_color) & (row['fill'] == green_color):
                name = 'Open > Close > Prev Close'
                current_candle = 'green_fill'
            elif (row['color'] == green_color) & (row['fill'] == green_fill_color_hollow):
                name = 'Prev Close < Close > Open'
                current_candle = 'green_hollow'
            elif (row['color'] == red_color) & (row['fill'] == red_color):
                name = 'Open > Close < Prev Close'
                current_candle = 'red_fill'
            elif (row['color'] == red_color) & (row['fill'] == red_fill_color_hollow):
                name = 'Prev Close > Close > Open'
                current_candle = 'red_hollow'
            else:
                name = 'Hollow Candles'

            
            # Make sure each candle type appears only once in the legend
            if (not shown_green_fill) & (current_candle == 'green_fill'):
                showlegend = True
                shown_green_fill = True
            elif (not shown_green_hollow) & (current_candle == 'green_hollow'):
                showlegend = True
                shown_green_hollow = True
            elif (not shown_red_fill) & (current_candle == 'red_fill'):
                showlegend = True
                shown_red_fill = True
            elif (not shown_red_hollow) & (current_candle == 'red_hollow'):
                showlegend = True
                shown_red_hollow = True
            else:
                showlegend = False
        
            color_dict = dict(
                fillcolor = row['fill'],
                line=dict(color = row['color'])
            )
            
            fig.add_trace(
                go.Candlestick(
                    x = [row['Date']],
                    open = [row['Open']],
                    high = [row['High']],
                    low = [row['Low']],
                    close = [row['Close']],
                    name = name,                    
                    increasing = color_dict,
                    decreasing = color_dict,
                    showlegend = showlegend,
                    legendgroup = f'{target_deck}',
                    legendgrouptitle = legendgrouptitle
                ),
                row = target_deck, col = 1
            )
        
    # Update layout and axes
    if add_title:
        fig.update_layout(
            title = dict(
                text = title,
                font_size = title_font_size,
                y = 0.98,
                x = 0.45,
                xanchor = 'center',
                yanchor = 'top'
            )
        )
    fig.update_xaxes(
        rangeslider_visible=False
    )
    fig.update_yaxes(
        range = (y_min, y_max),
        title = f'Price',
        # nticks = n_yticks_max,        
        showticklabels = True,
        row = target_deck, col = 1
    )

    if deck_type in ['double', 'triple']:
        legend_tracegroupgap = adjust_legend_position(fig_data, deck_type)
        fig.update_layout(
            legend_tracegroupgap = legend_tracegroupgap,
            legend_traceorder = 'grouped'
        )

    print(f'Candlestick legend_tracegroupgap = {legend_tracegroupgap}')

    fig_data.update({'fig': fig})
    fig_data['y_min'].update({target_deck: y_min})
    fig_data['y_max'].update({target_deck: y_max})

    return fig_data

In [63]:
def add_hist_price(
    fig_data,
    df_price,
    tk,
    target_deck = 1,
    plot_type = 'scatter',
    n_yticks_max = 16,
    price_type = 'adjusted close',
    add_title = True,
    title = None,
    title_font_size = 32,
    theme = 'dark',
    color_theme = None,
    fill_below = False
):
    """
    fig_data:
        template to add the plot to
    target_deck:
        1 (upper), 2 (second from top), 3 (third from top)
    plot_type:
        'scatter' or 'bar'
    price_type:
        one of 'adjusted close', 'close', 'open', 'high', 'low', 'volume', 'dollar volume'

    """

    if isinstance(df_price, pd.Series):
        df_tk = df_price.copy()
    elif isinstance(df_price, pd.DataFrame):
        df_tk = df_price[tk]
    else:
        print('Incorrect format of input data')
        exit

    legend_name = price_type.title()
    yaxis_title = price_type.title()
    if add_title & (title is None):
        title = f'{tk} {price_type.title()}'

    fig = fig_data['fig']
    fig_y_min = fig_data['y_min'][target_deck]
    fig_y_max = fig_data['y_max'][target_deck]
    deck_type = fig_data['deck_type']

    style = theme_style[theme]

    if color_theme is None:
        linecolor = style['basecolor']
    else:
        color_idx = style['overlay_color_selection'][color_theme][1][0]
        linecolor = style['overlay_color_theme'][color_theme][color_idx]
    
    # Must be specified as RGBA
    if plot_type == 'bar':
        opacity = 0.9
    else:
        opacity = 0.6

    fillcolor = linecolor.replace(', 1)', f', {opacity})')

    print(linecolor, fillcolor)

    if fill_below:
        fill = 'tozeroy'
    else:
        fill = 'none'

    min_y = min(df_tk)
    max_y = max(df_tk)
    y_min, y_max = set_axis_limits(min_y, max_y)

    if fig_y_min is not None:
        y_min = min(fig_y_min, y_min)
    if fig_y_max is not None:
        y_max = max(fig_y_max, y_max)

    if target_deck > 1:
        y_max *= 0.999

    legendgrouptitle = {}
    if deck_type == 'triple':
        legendtitle = tripledeck_legendtitle[target_deck]
        legendgrouptitle = dict(
            text = legendtitle,
            font_size = 16,
            font_weight = 'bold'
        )

    # Add trace
    if plot_type == 'bar':
        fig.add_trace(
            go.Bar(
                x = df_tk.index.astype(str),
                y = df_tk,
                marker_color = fillcolor,
                width = 1,
                name = legend_name,
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                showlegend = True
            ),
            row = target_deck, col = 1
        )

    else:
        fig.add_trace(
            go.Scatter(
                x = df_tk.index.astype(str),
                y = df_tk,
                line_color = linecolor,
                fill = fill,
                fillcolor = fillcolor,
                showlegend = True,
                name = legend_name,
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle
            ),
            row = target_deck, col = 1
        )

    # Update layout and axes
    if add_title:
        fig.update_layout(
            title = dict(
                text = title,
                font_size = title_font_size,
                y = 0.98,
                x = 0.45,
                xanchor = 'center',
                yanchor = 'top'
            )
        )

    fig.update_yaxes(
        range = (y_min, y_max),
        title = yaxis_title,
        showticklabels = True,
        nticks = n_yticks_max,
        row = target_deck, col = 1
    )

    if deck_type in ['double', 'triple']:
        legend_tracegroupgap = adjust_legend_position(fig_data, deck_type)
        fig.update_layout(
            legend_tracegroupgap = legend_tracegroupgap,
            legend_traceorder = 'grouped'
        )

    print(f'legend_tracegroupgap = {legend_tracegroupgap}')

    fig_data.update({'fig': fig})
    fig_data['y_min'].update({target_deck: y_min})
    fig_data['y_max'].update({target_deck: y_max})

    return fig_data

In [23]:
def stochastic_oscillator(
    close_tk,
    high_tk,
    low_tk,
    fast_k_period = 14,
    smoothing_period = 3,
    sma_d_period = 3,
    stochastic_type = 'Slow'
):
    """
    stochastic_type: 'Fast', 'Slow', 'Full'
    NOTES:
    1) fast_k_period is also know as the look--back period
    2) smoothing_period is the period used in slow %K and full %K
    3) sma_d_period is the %D averaging period used in fast, slow and full stochastics
    4) if sma_d_period == smoothing_period, then the slow and full stochastics become equivalent
    
    """
    fast_low = low_tk.rolling(window = fast_k_period, min_periods = 1).min()
    fast_high = high_tk.rolling(window = fast_k_period, min_periods = 1).max()
    fast_k_line = 100 * (close_tk - fast_low) / (fast_high - fast_low)

    if stochastic_type.lower() == 'fast':
        
        k_line = fast_k_line.copy()    
        d_line = k_line.rolling(window = sma_d_period, min_periods = 1).mean()
        stochastic_label = f'({fast_k_period}, {sma_d_period})'
        stochastic_type = 'Fast'

    elif (stochastic_type.lower() == 'full') | (sma_d_period != smoothing_period):
        
        k_line = fast_k_line.rolling(window = smoothing_period, min_periods = 1).mean()
        d_line = k_line.rolling(window = sma_d_period, min_periods = 1).mean()
        stochastic_label = f'({fast_k_period}, {smoothing_period}, {sma_d_period})'
        stochastic_type = 'Full'

    else:
        # This includes the case of 
        # (stochastic_type == 'slow') | (sma_d_period == smoothing_period)
        # and any other stochastic_type specified.
        
        k_line = fast_k_line.rolling(window = smoothing_period, min_periods = 1).mean()
        d_line = k_line.rolling(window = sma_d_period, min_periods = 1).mean()
        stochastic_label = f'({fast_k_period}, {sma_d_period})'
        stochastic_type = 'Slow'

    k_line.index = k_line.index.astype(str)
    d_line.index = d_line.index.astype(str)

    stochastic_data = {
        'k_line': k_line,
        'd_line': d_line,
        'label': stochastic_label,
        'type': stochastic_type
    }

    return stochastic_data

In [24]:
fast_k_period = 14
smoothing_period = 3
sma_d_period = 5

x_min = datetime(2024, 3, 21)
x_max = datetime(2024, 9, 21)

stochastic_data = stochastic_oscillator(close_tk, high_tk, low_tk)
"""
stochastic_data = stochastic_oscillator(
    close_tk[x_min: x_max],
    high_tk[x_min: x_max],
    low_tk[x_min: x_max],
    fast_k_period = fast_k_period,
    smoothing_period = smoothing_period,
    sma_d_period = sma_d_period
)
"""

'\nstochastic_data = stochastic_oscillator(\n    close_tk[x_min: x_max],\n    high_tk[x_min: x_max],\n    low_tk[x_min: x_max],\n    fast_k_period = fast_k_period,\n    smoothing_period = smoothing_period,\n    sma_d_period = sma_d_period\n)\n'

In [62]:
def add_stochastic(
    fig_data,
    stochastic_data,
    tk,
    target_deck = 2,
    oversold_threshold = 20,
    overbought_threshold = 80,
    add_threshold_overlays = True,
    n_yticks_max = 6,
    add_title = False,
    title_font_size = 32,
    theme = 'dark'
):
    """
    stochastic_data: output from stochastic_oscillator()
    tk: ticker for which to plot the stochastic %K and %D lines

    """

    fig_stochastic = fig_data['fig']
    deck_type = fig_data['deck_type']

    k_line = stochastic_data['k_line']
    d_line = stochastic_data['d_line']
    stochastic_label = stochastic_data['label']
    stochastic_type = stochastic_data['type']
    
    style = theme_style[theme]

    title_stochastic = f'{tk} {stochastic_label} {stochastic_type} Stochastic Oscillator (%)'
    yaxis_title = f'Stochastic (%)'

    y_max = 99.99 if target_deck > 1 else 100

    legendgrouptitle = {}
    if deck_type == 'triple':
        legendtitle = tripledeck_legendtitle[target_deck]
        legendgrouptitle = dict(
            text = legendtitle,
            font_size = 16,
            font_weight = 'bold'
        )

    fig_stochastic.add_trace(
        go.Scatter(
            x = k_line.index,
            y = k_line,
            line_color = style['kline_linecolor'],
            line_width = 2,
            legendgroup = f'{target_deck}',
            legendgrouptitle = legendgrouptitle,
            name = f'{stochastic_type} %K Line'
        ),
        row = target_deck, col = 1
    )
    fig_stochastic.add_trace(
        go.Scatter(
            x = d_line.index,
            y = d_line,
            line_color = style['dline_linecolor'],
            line_width = 2,
            legendgroup = f'{target_deck}',
            legendgrouptitle = legendgrouptitle,
            name = f'{stochastic_type} %D Line'
        ),
        row = target_deck, col = 1
    )

    if add_threshold_overlays:

        stochastic_hlines = pd.DataFrame(
            {
                'oversold': oversold_threshold,
                'overbought': overbought_threshold,
                'y_max': y_max
            },
            index = k_line.index
        )
        fig_stochastic.add_trace(
            go.Scatter(
                x = stochastic_hlines.index,
                y = stochastic_hlines['y_max'],
                line_color = 'black',
                line_width = 0,
                hoverinfo = 'skip',
                showlegend = False
            ),
            row = target_deck, col = 1
        )
        fig_stochastic.add_trace(
            go.Scatter(
                x = stochastic_hlines.index,
                y = stochastic_hlines['overbought'],
                line_color = style['overbought_linecolor'],
                line_width = 2,
                fill = 'tonexty',  # fill to previous scatter trace
                fillcolor = style['overbought_fillcolor'],
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                name = f'Overbought > {overbought_threshold}%'
            ),
            row = target_deck, col = 1
        )

        fig_stochastic.add_trace(
            go.Scatter(
                x = stochastic_hlines.index,
                y = stochastic_hlines['oversold'],
                line_color = style['oversold_linecolor'],
                line_width = 2,
                fill = 'tozeroy',
                fillcolor = style['oversold_fillcolor'],
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                name = f'Oversold < {oversold_threshold}%'
            ),
            row = target_deck, col = 1
        )

    # Update layout and axes
    if add_title:
        fig_stochastic.update_layout(
            title = dict(
                text = title_stochastic,
                font_size = title_font_size,
                y = 0.98,
                x = 0.45,
                xanchor = 'center',
                yanchor = 'top'
            )
        )

    fig_stochastic.update_yaxes(
        range = (0, y_max),
        title = yaxis_title,
        showticklabels = True,
        nticks = n_yticks_max,
        row = target_deck, col = 1
    )

    if deck_type in ['double', 'triple']:
        legend_tracegroupgap = adjust_legend_position(fig_data, deck_type)
        fig_stochastic.update_layout(
            legend_tracegroupgap = legend_tracegroupgap,
            legend_traceorder = 'grouped+reversed'
        )

    fig_data.update({'fig': fig_stochastic})
    fig_data['y_min'].update({target_deck: 0})
    fig_data['y_max'].update({target_deck: y_max})

    return fig_data


In [36]:
def bollinger_bands(
    prices,
    window = 20,
    n_std = 2.0,
    n_bands = 1
):
    """
    prices:
        series of ticker prices ('adjusted close', 'open', 'high', 'low' or 'close')
    window:
        size of the rolling window in days, defaults to 20
    n_std:
        width of the upper and lower bands in standard deviations, defaults to 2.0
    n_bands:
        number of pairs of bands to be created, defaults to 1, max 3

    Returns a list of bollinger band dictionaries
    """

    eps = 1e-6

    n_bands = min(3, n_bands)

    df_sma = prices.rolling(window = window, min_periods = 1).mean()
    df_std = prices.rolling(window = window, min_periods = 1).std(ddof=0)
    
    bollinger_list = [{
        'data': df_sma,
        'name': f'SMA {window}',
        'idx_offset': 0,
        'showlegend': True
    }]

    k = 0
    # k = 0 if each band_width is an integer within the accuray of eps    
    for i in range(n_bands + 1)[1:]:
        band_width = i * n_std
        if abs(float(int(band_width)) - band_width) > eps:
            k = 1
            break

    for i in range(n_bands + 1)[1:]:
        
        band_width = i * n_std

        upper_band = df_sma + band_width * df_std
        upper_name = f'({window}, {band_width:.{k}f}) Upper Bollinger'
        bollinger_list.append({
            'data': upper_band,
            'name': upper_name,
            'idx_offset': i,
            'showlegend': True
        })

        lower_band = df_sma - band_width * df_std        
        lower_name = f'({window}, {band_width:.{k}f}) Lower Bollinger'
        bollinger_list.append({
            'data': lower_band,
            'name': lower_name,
            'idx_offset': -i,
            'showlegend': True
        })

    bollinger_list = sorted(bollinger_list, key = itemgetter('idx_offset'), reverse = True)

    return bollinger_list

In [40]:
def ma_envelopes(
    prices,
    ma_type = None,
    window = 20,
    prc_offset = 5,
    n_bands = 3
):
    """
    prices:
        series of ticker prices ('adjusted close', 'open', 'high', 'low' or 'close')
    ma_type:
        one of 'sma', 'ema', dema', tema'
    window:
        size of the rolling window in days
    prc_offset: 
        vertical offset from base moving average in percentage points (-99% to 99%)
    n_bands:
        number of pairs of envelopes to be created, defaults to 3 (max)

    Returns a list of ma envelope dictionaries
    """

    eps = 1e-6
    
    if ma_type is None:
        ma_type = 'sma'

    n_bands = min(3, n_bands)
    if abs(prc_offset) > 99:
        prc_offset = math.sign(prc_offset) * 99

    base_ma = moving_average(prices, ma_type, window)

    base_name = f'{ma_type.upper()} {window}'
    
    ma_envelope_list = [{
        'data': base_ma,
        'name': base_name,
        'idx_offset': 0,
        'showlegend': True
    }]

    k = 0
    # k = 0 if each ma_offset is an integer within the accuray of eps    
    for i in range(n_bands + 1)[1:]:
        ma_offset = i * prc_offset
        if abs(float(int(ma_offset)) - ma_offset) > eps:
            k = 1
            break

    for i in range(n_bands + 1)[1:]:
        
        ma_offset = i * prc_offset

        upper_band = base_ma * (1 + ma_offset / 100)
        upper_name = f'({window}, {ma_offset:.{k}f}%) Upper Envelope'
        ma_envelope_list.append({
            'data': upper_band,
            'name': upper_name,
            'idx_offset': i,
            'showlegend': True
        })

        lower_band = base_ma * (1 - ma_offset / 100)
        lower_name = f'({window}, {ma_offset:.{k}f}%) Lower Envelope'
        ma_envelope_list.append({
            'data': lower_band,
            'name': lower_name,
            'idx_offset': -i,
            'showlegend': True
        })

    ma_envelope_list = sorted(ma_envelope_list, key = itemgetter('idx_offset'), reverse = True)

    return ma_envelope_list

In [69]:
def add_bollinger_overlays(
    fig_data,
    bollinger_list,
    target_deck = 1,
    x_min = None,
    x_max = None,
    theme = 'dark',
    color_theme = 'gold'
):
    """
    df_price: df_close or df_adj_close, depending on the underlying figure in fig_data

    """

    x_min = start_date if x_min is None else x_min
    x_max = end_date if x_max is None else x_max

    deck_type = fig_data['deck_type']

    n_boll = int((len(bollinger_list) + 1) / 2)
    # print(n_boll)

    style = theme_style[theme]
    overlay_color_idx = style['overlay_color_selection'][color_theme][n_boll]
    
    current_names = [tr['name'] for tr in fig_data['fig']['data'] if (tr['legendgroup'] == str(target_deck))]

    bollinger_overlays = []
    
    for boll in bollinger_list:
        
        if boll['name'] not in current_names:
            bollinger_overlays.append({
                'data': boll['data'][x_min: x_max],
                'name': boll['name'],
                'color_idx': overlay_color_idx[abs(boll['idx_offset'])],
                'showlegend': boll['showlegend']
            })

    # color_map = fig_data['color_map']
    color_map = {}

    for overlay in bollinger_overlays:
        fig_data = add_overlay(
            fig_data,
            overlay['data'],
            overlay['name'],
            overlay['color_idx'],
            overlay['showlegend'],
            target_deck = target_deck,
            theme = theme,
            color_theme = color_theme
        )
        color_map.update({overlay['name']: overlay['color_idx']})

    if deck_type in ['double', 'triple']:
        legend_tracegroupgap = adjust_legend_position(fig_data, deck_type)
        fig_data['fig'].update_layout(
            legend_tracegroupgap = legend_tracegroupgap,
            legend_traceorder = 'grouped'
        )
        
    fig_data.update({'color_map': color_map})

    return fig_data

In [70]:
def add_ma_envelope_overlays(
    fig_data,
    ma_envelope_list,
    target_deck = 1,    
    x_min = None,
    x_max = None,
    theme = 'dark',
    color_theme = 'gold'
):
    """
    """
    
    x_min = start_date if x_min is None else x_min
    x_max = end_date if x_max is None else x_max

    deck_type = fig_data['deck_type']

    n_env = int((len(ma_envelope_list) + 1) / 2)

    style = theme_style[theme]
    overlay_color_idx = style['overlay_color_selection'][color_theme][n_env]
    
    current_names = [tr['name'] for tr in fig_data['fig']['data'] if (tr['legendgroup'] == str(target_deck))]

    ma_envelope_overlays = []
    
    for env in ma_envelope_list:
        
        if env['name'] not in current_names:
            ma_envelope_overlays.append({
               'data': env['data'][x_min: x_max],
               'name': env['name'],
               'color_idx': overlay_color_idx[abs(env['idx_offset'])],
               'showlegend': env['showlegend']
            })

    color_map = {}

    for overlay in ma_envelope_overlays:
        fig_data = add_overlay(
            fig_data,
            overlay['data'],
            overlay['name'],
            overlay['color_idx'],
            overlay['showlegend'],
            target_deck = target_deck,
            theme = theme,
            color_theme = color_theme
        )
        color_map.update({overlay['name']: overlay['color_idx']})
    
    if deck_type in ['double', 'triple']:
        legend_tracegroupgap = adjust_legend_position(fig_data, deck_type)
        fig_data['fig'].update_layout(
            legend_tracegroupgap = legend_tracegroupgap,
            legend_traceorder = 'grouped'
        )

    fig_data.update({'color_map': color_map})

    return fig_data

In [51]:
# From https://github.com/matplotlib/mplfinance/blob/master/examples/indicators/rsi.py

def relative_strength(prices, period = 14):
    """
    http://stockcharts.com/school/doku.php?id=chart_school:glossary_r#relativestrengthindex
    http://www.investopedia.com/terms/r/rsi.asp
    """
    
    deltas = np.diff(prices)
    seed = deltas[:period + 1]
    up = seed[seed >= 0].sum() / period
    down = -seed[seed < 0].sum() / period
    rs = up / down
    array_rsi = np.zeros_like(prices)
    array_rsi[:period] = 100. - 100. / (1. + rs)

    for i in range(period, len(prices)):
        delta = deltas[i - 1]  # cause the diff is 1 shorter

        if delta > 0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up * (period - 1) + upval) / period
        down = (down * (period - 1) + downval) / period

        rs = up / down
        array_rsi[i] = 100. - 100. / (1. + rs)

    rsi = pd.Series(data = array_rsi, index = prices.index.astype(str))
    rsi_type = f'{period}'

    rsi_data = {
        'rsi': rsi,
        'type': rsi_type
    }

    return rsi_data

In [60]:
def add_rsi(
    fig_data,
    rsi_data,
    tk,
    target_deck = 2,
    oversold_threshold = 30,
    overbought_threshold = 70,
    add_threshold_overlays = True,
    n_yticks_max = 6,
    add_title = False,
    title_font_size = 32,
    theme = 'dark'
):
    """
    rsi_data:   output from relative_strength()
    tk:         ticker for which to plot RSI
    price_type: normally 'adjusted close' or 'close', whatever the RSI is based on
    df_price:   dataframe/series of prices to overlay (if overlay_price is True)

    """

    rsi = rsi_data['rsi']
    rsi_type = rsi_data['type']

    fig_rsi = fig_data['fig']    
    deck_type = fig_data['deck_type']

    style = theme_style[theme]

    title_rsi = f'{tk} Relative Strength Index {rsi_type} (%)'
    yaxis_title = f'RSI (%)'

    y_max = 99.99 if target_deck > 1 else 100
    
    legendgrouptitle = {}
    if deck_type == 'triple':
        legendtitle = tripledeck_legendtitle[target_deck]
        legendgrouptitle = dict(
            text = legendtitle,
            font_size = 16,
            font_weight = 'bold'
        )

    fig_rsi.add_trace(
        go.Scatter(
            x = rsi.index,
            y = rsi,
            line_color = style['rsi_linecolor'],
            line_width = 2,
            legendgroup = f'{target_deck}',
            legendgrouptitle = legendgrouptitle,            
            name = f'RSI {rsi_type} (%)'
        ),
        row = target_deck, col = 1
    )

    if add_threshold_overlays:
        rsi_hlines = pd.DataFrame(
            {
                'oversold': oversold_threshold,
                'overbought': overbought_threshold,
                'y_max': y_max
            },
            index = rsi.index
        )
        fig_rsi.add_trace(
            go.Scatter(
                x = rsi_hlines.index,
                y = rsi_hlines['y_max'],
                line_color = 'black',
                line_width = 0,
                hoverinfo = 'skip',                
                showlegend = False
            ),
            row = target_deck, col = 1
        )
        fig_rsi.add_trace(
            go.Scatter(
                x = rsi_hlines.index,
                y = rsi_hlines['overbought'],
                line_color = style['overbought_linecolor'],
                line_width = 2,
                fill = 'tonexty',  # fill to previous scatter trace
                fillcolor = style['overbought_fillcolor'],
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                name = f'Overbought > {overbought_threshold}%'
            ),
            row = target_deck, col = 1
        )
        fig_rsi.add_trace(
            go.Scatter(
                x = rsi_hlines.index,
                y = rsi_hlines['oversold'],
                line_color = style['oversold_linecolor'],
                line_width = 2,
                fill = 'tozeroy',
                fillcolor = style['oversold_fillcolor'],
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                name = f'Oversold < {oversold_threshold}%'
            ),
            row = target_deck, col = 1
        )

    # Update layout and axes
    if add_title:
        fig_rsi.update_layout(
            title = dict(
                text = title_rsi,
                font_size = title_font_size,
                y = 0.98,
                x = 0.45,
                xanchor = 'center',
                yanchor = 'top'
            )
        )
    
    fig_rsi.update_yaxes(
        range = (0, y_max),
        title = yaxis_title,
        showticklabels = True,
        nticks = n_yticks_max,
        row = target_deck, col = 1
    )

    if deck_type in ['double', 'triple']:
        legend_tracegroupgap = adjust_legend_position(fig_data, deck_type)
        fig_rsi.update_layout(
            legend_tracegroupgap = legend_tracegroupgap,
            legend_traceorder = 'grouped+reversed'
        )

    fig_data.update({'fig': fig_rsi})
    fig_data['y_min'].update({target_deck: 0})
    fig_data['y_max'].update({target_deck: y_max})

    return fig_data

In [ ]:
# These must be specified by the user, except signal_type and signal_window if the user chooses not to add signal
# The price types should be capitalized as they appear in the menu, must also be consistent with price_list
# The MA acronyms can stay lower case, easy to convert 

# Should add more options such as stochastic's K and D lines, also 'wwma'

diff_data = {
    'p_base': 'close',  # 'adjusted close', 'adj close', 'close', 'open', 'high', 'low'
    'p1_type': 'ema',  # 'adjusted close', 'adj close', 'close', 'open', 'high', 'low', 'sma', 'ema', 'dema', 'tema', 'wma', 'wwma', 'k-line', 'd-line'
    'p2_type': 'wma',  # 'adjusted close', 'adj close', 'close', 'open', 'high', 'low', 'sma', 'ema', 'dema', 'tema', 'wma', 'wwma', 'k-line', 'd-line'
    'p1_window': 10,
    'p2_window': 10,
    'signal_type': 'ema',  # 'sma', 'ema', 'dema', 'tema', 'wma', 'wwma'
    'signal_window': 5
}

In [80]:
def plot_diff_plotly(
    tk,
    diff_data,
    price_type_map,
    reverse_diff = False,
    add_signal = True,
    n_ticks_max = 48,
    n_yticks_max = 16,
    plot_width = 1450,
    plot_height = 750,
    title_font_size = 32,
    theme = 'dark'
):
    """
    price_type_map = {
        'Adj Close': adj_close_tk,
        'Adjusted Close': adj_close_tk,
        'Close': close_tk,
        'Open': open_tk,
        'High': high_tk,
        'Low': low_tk
    }
    reverse_diff:
        if True, the (p2 - p1) difference will be used instead of (p1 - p2)
    add_signal:
        if True, a signal will be added that is a moving average of the calculated difference
    """
    
    base = diff_data['p_base']
    p_base_name = base.title()
    p_base = price_type_map[p_base_name]

    p1_type = diff_data['p1_type']
    p2_type = diff_data['p2_type']
    p1_window = diff_data['p1_window']
    p2_window = diff_data['p2_window']
    signal_type = diff_data['signal_type']
    signal_window = diff_data['signal_window']
    
    price_types = ['adjusted close', 'adj close', 'close', 'open', 'high', 'low']
    ma_types = ['sma', 'ema', 'dema', 'tema', 'wma']

    if p1_type in price_types:
        p1_name = 'Adjusted Close' if p1_name == 'adj close' else p1_type.title()
        try:
            p1 = price_type_map[p1_name]
        except:
            p1 = price_type_map['Adj Close']

    elif p1_type in ma_types:
        p1 = moving_average(p_base, p1_type, p1_window)
        p1_name = f'{p1_type.upper()} {p1_window}'

    if p2_type in price_types:
        p2_name = 'Adjusted Close' if p2_name == 'adj close' else p2_type.title()
        try:
            p2 = price_type_map[p2_name]
        except:
            p2 = price_type_map['Adj Close']

    elif p2_type in ma_types:
        p2 = moving_average(p_base, p2_type, p2_window)
        p2_name = f'{p2_type.upper()} {p2_window}'

    if not reverse_diff:
        diff = p1 - p2
        diff_title = f'{tk} {p_base_name} {p1_name} - {p2_name} Oscillator'
        diff_positive_name = f'{p1_name} > {p2_name}'
        diff_negative_name = f'{p1_name} < {p2_name}'
    else:
        diff = p2 - p1
        diff_title = f'{tk} {p_base_name} {p2_name} - {p1_name} Oscillator'
        diff_positive_name = f'{p2_name} > {p1_name}'
        diff_negative_name = f'{p2_name} < {p1_name}'

    diff_signal = moving_average(diff, signal_type, signal_window)
    signal_name = f'Diff {signal_type.upper()} {signal_window} Signal'

    x_min = str(diff.index.min().date())
    x_max = str(diff.index.max().date())
 
    min_diff = min(diff)
    max_diff = max(diff)
    
    y_diff_min, y_diff_max = set_axis_limits(min_diff, max_diff)
    
    diff_positive = diff.copy()
    diff_negative = diff.copy()

    prev_v = diff.iloc[0]
    diff_positive.iloc[0] = prev_v if prev_v >= 0 else np.nan
    diff_negative.iloc[0] = prev_v if prev_v < 0 else np.nan

    for idx in diff.index[1:]:
        
        curr_v = diff.loc[idx]

        if np.sign(curr_v) != np.sign(prev_v):
            # Set both diff copies to 0 if the value is changing sign
            diff_positive[idx] = 0
            diff_negative[idx] = 0
        else:
            # Set both diff copies to current value or NaN
            diff_positive[idx] = curr_v if curr_v >= 0 else np.nan
            diff_negative[idx] = curr_v if curr_v < 0 else np.nan
        
        prev_v = curr_v

    style = theme_style[theme]
    
    fig_diff = make_subplots(rows = 1, cols = 1)

    fig_diff.add_trace(
        go.Scatter(
            x = diff_positive.index.astype(str),
            y = diff_positive,
            line_color = style['diff_green_linecolor'],
            line_width = 2,
            fill = 'tozeroy',
            fillcolor = style['diff_green_fillcolor'],
            name = diff_positive_name
        )
    )
    fig_diff.add_trace(
        go.Scatter(
            x = diff_negative.index.astype(str),
            y = diff_negative,
            line_color = 'darkred',
            line_width = 2,
            fill = 'tozeroy',
            fillcolor = style['diff_red_fillcolor'],
            name = diff_negative_name
        )
    )
    if add_signal:
        fig_diff.add_trace(
            go.Scatter(
                x = diff_signal.index.astype(str),
                y = diff_signal,
                line_color = style['signal_color'],
                line_width = 2,
                name = signal_name
            )
        )
    
    # Add plot border
    fig_diff.add_shape(
        type = 'rect',
        xref = 'x',  # use 'x' because of seconday axis - 'paper' does not work correctly
        yref = 'paper',
        x0 = x_min,
        x1 = x_max,
        y0 = 0,
        y1 = 1,
        line_color = style['x_linecolor'],
        line_width = 0.3
    )
    # Update layout and axes
    fig_diff.update_layout(
        width = plot_width,
        height = plot_height,
        xaxis_rangeslider_visible = False,
        template = style['template'],
        title = dict(
            text = diff_title,
            font_size = title_font_size,
            y = 0.95,
            x = 0.45,
            xanchor = 'center',
            yanchor = 'top'
        )
    )
    fig_diff.update_yaxes(
        title_text = f'Oscillator',
        range = (y_diff_min, y_diff_max),
        secondary_y = False,
        nticks = n_yticks_max,
        gridcolor = style['y_gridcolor'],
        ticks = 'outside',
        ticklen = 8,
        ticklabelshift = 5,  # not working
        ticklabelstandoff = 10,  # not working
    )
    fig_diff.update_xaxes(
        type = 'category',
        nticks = n_ticks_max,
        tickangle = -90,
        gridcolor = style['x_gridcolor'],
        ticks = 'outside',
        ticklen = 8,
        ticklabelshift = 5,  # not working
        ticklabelstandoff = 10,  # not working
        showgrid = True
    )

    return fig_diff

In [411]:
tk = 'AAPL'
theme = 'dark'

print(str(min(close_tk.index)))

# fig_close_data = plot_hist_plotly(close_tk, tk, price_type = 'close', theme = theme)
# fig_close = fig_close_data['fig']

macd_data = get_macd(close_tk)
macd_v_data = get_macd_v(close_tk, high_tk, low_tk)

atr_data = average_true_rate(close_tk, high_tk, low_tk, n = 26)
atr = atr_data['atr']
atr.index = atr.index.astype(str)
atrp = atr_data['atrp']
atr_list = [{'name': 'ATR 26', 'data': atr, 'show': True}]

bollinger_list = bollinger_bands(close_tk, window = 20, n_std = 1, n_bands = 3)

ma_envelope_list = ma_envelopes(close_tk, window = 20, prc_offset = 5, n_bands = 2)

rsi_data = relative_strength(close_tk)

date_index = close_tk.index

deck_type = 'triple'
# deck_type = 'double'

theme = 'dark'
# theme = 'light'
color_theme = 'magenta'
# color_theme = 'tableau'
color_theme_2 = 'sapphire'
color_theme_vol = 'rainbow'
color_theme_vol_2 = 'sapphire'

fig_data = create_template(
    date_index,
    deck_type = deck_type,
    plot_width = 1280, 
    plot_height_1 = 650,
    plot_height_2 = 180,
    plot_height_3 = 180,
    # n_ticks_max = 52
    theme = theme
)
##### NOTE: Decks need to get populated from top to bottom, i.e. from 1 to 3, otherwise the legends will end up in the wrong order

fig_data = add_candlestick(fig_data, ohlc_tk, tk, candle_type = 'traditional', target_deck = 1, theme = theme)
# fig_data = add_candlestick(fig_data, ohlc_tk, tk, candle_type = 'hollow', target_deck = 1, theme = theme)

# fig_data = add_hist_price(fig_data, close_tk, tk, target_deck = 1, price_type = 'close', theme = theme)

fig_data = add_ma_overlays(fig_data, close_tk, ema_list[: 6], target_deck = 1, theme = theme, color_theme = color_theme)
fig_data = add_ma_overlays(fig_data, close_tk, ma_list[: 6], target_deck = 1, theme = theme, color_theme = color_theme_2)

# fig_data = add_bollinger_overlays(fig_data, bollinger_list, target_deck = 1, theme = theme)

# fig_data = add_ma_envelope_overlays(fig_data, ma_envelope_list, target_deck = 1, theme = theme, color_theme = 'gold')

# fig_data = add_hist_price(fig_data, close_tk, tk, target_deck = 2, add_title = False, n_yticks_max = 4, price_type = 'close', theme = theme)
# fig_data = add_ma_envelope_overlays(fig_data, ma_envelope_list, target_deck = 2, theme = theme, color_theme = 'gold')
#fig_data = add_ma_overlays(fig_data, close_tk, ema_list[: 6], target_deck = 2, theme = theme, color_theme = color_theme_2)
# print(fig_data['color_map'])

# fig_data = add_hist_price(fig_data, close_tk, tk, n_yticks_max = 7, target_deck = 2, price_type = 'close', theme = theme, color_theme = color_theme_2)
# fig_data = add_candlestick(fig_data, df_ohlc, tk, candle_type = 'traditional', target_deck = 2)
# fig_data = add_price_overlays(fig_data, price_list, tk, target_deck = 2, add_yaxis_title = True, theme = theme, color_theme = color_theme_2)

# fig_data = add_macd(fig_data, close_tk, macd_data, n_yticks_max = 7, volatility_normalized = False, histogram_type = 'macd-zero', include_signal = False, target_deck = 2, theme = theme)
# fig_data = add_price_overlays(fig_data, atr_list, tk, target_deck = 2, theme = theme, color_theme = color_theme_2)

fig_data = add_macd(fig_data, close_tk, macd_v_data, n_yticks_max = 7, volatility_normalized = True, histogram_type = 'macd-signal', include_signal = False, target_deck = 3, theme = theme)

# fig_data = add_stochastic(fig_data, stochastic_data, tk, target_deck = 2, add_threshold_overlays = True, theme = theme)
# fig_data = add_rsi(fig_data, rsi_data, tk, target_deck = 2, add_threshold_overlays = True, theme = theme)
"""
fig_data = add_hist_price(
    fig_data,
    volume_tk,
    tk,
    target_deck = 3,
    plot_type = 'bar',
    n_yticks_max = 7,
    price_type = 'volume',
    add_title = False,
    theme = theme,
    color_theme = color_theme_vol_2,
    fill_below = True
)
"""
# fig_data = add_candlestick(fig_data, df_ohlc, tk, candle_type = 'traditional', target_deck = 3)
"""
fig_data = add_hist_price(
    fig_data,
    volume_tk,
    tk,
    target_deck = 3,
    plot_type = 'bar',
    n_yticks_max = 7,
    price_type = 'volume',
    add_title = False,
    theme = theme,
    color_theme = color_theme_vol_2,
    fill_below = True
)
"""
# fig_data = add_hist_price(fig_data, volume_tk, tk, target_deck = 3, price_type = 'volume', theme = theme, fill_below = True)

# fig_data = add_price_overlays(fig_data, atr_list, tk, target_deck = 2, theme = theme, color_theme = color_theme_2)

print('Plot heights:')
if deck_type == 'double':
    print(fig_data['plot_height'][1], fig_data['plot_height'][2])
elif deck_type == 'triple':
    print(fig_data['plot_height'][1], fig_data['plot_height'][2], fig_data['plot_height'][3])
else:
    print(fig_data['plot_height'][1])
    
fig = fig_data['fig']
fig.show()

# Add MACD-V and ATR line

2023-10-03 00:00:00
n_traces_upper, n_traces_middle, n_traces_lower
2 0 0
488.6942
Candlestick legend_tracegroupgap = 488.6942
n_traces_upper, n_traces_middle, n_traces_lower
8 0 0
374.6942
n_traces_upper, n_traces_middle, n_traces_lower
14 0 0
260.6942
n_traces_upper, n_traces_middle, n_traces_lower
14 0 2
409.3586
Plot heights:
650 180 180


In [27]:
# n_decks = max([trace['legendgroup'] for trace in fig_data['fig']['data'] if trace['legendgroup'].isdigit()])
# print(n_decks)
# print(fig_data['fig']['data'])
print(fig_data['color_map'])


{'EMA 10': 0, 'EMA 20': 1, 'EMA 30': 2, 'EMA 40': 3, 'EMA 50': 4, 'EMA 60': 5}


In [28]:
macd_data = get_macd(adj_close_tk)

In [29]:
x = fig_data['fig']['data'][1]
if x['showlegend'] is not None:
    cond = x['showlegend']
else:
    cond = True
print(cond)

False


Candlestick

In [235]:
def plot_candlestick_plotly(
    df_ohlc,
    tk,
    candle_type = 'hollow',
    n_ticks_max = 48,
    n_yticks_max = 16,
    plot_width = 1450,
    plot_height = 750,
    title_font_size = 32,
    theme = 'dark'
):
    """
    candle_type: 'hollow' or 'traditional'
        
    """
    
    style = theme_style[theme]

    # Colors must be in the RGBA format
    red_color = style['red_color']
    green_color = style['green_color']
    red_fill_color = red_color
    green_fill_color = green_color
    red_fill_color_hollow = red_color.replace(', 1)', ', 0.25)')
    green_fill_color_hollow = green_color.replace(', 1)', ', 0.25)')

    df = df_ohlc.copy()

    min_y = min(df['Low'])
    max_y = max(df['High'])
    y_min, y_max = set_axis_limits(min_y, max_y)

    df['Date'] = df.index.astype(str)
    x_min = df['Date'].min()
    x_max = df['Date'].max()

    fig = make_subplots(rows = 1, cols = 1)

    if candle_type == 'traditional':
        
        title = f'{tk} Prices - Traditional Candles'

        shown_green = False
        shown_red = False

        for _, row in df.iterrows():

            if row['Close'] >= row['Open']:
                color_dict = dict(
                    fillcolor = green_fill_color,
                    line = dict(color = green_color)
                )
                name = 'Close > Open'
                current_candle = 'green'
            else:
                color_dict = dict(
                    fillcolor = red_fill_color,
                    line = dict(color = red_color)
                )
                name = 'Open > Close'
                current_candle = 'red'

            # Make sure each candle type appears only once in the legend
            if (not shown_green) & (current_candle == 'green'):
                showlegend = True
                shown_green = True
            elif (not shown_red) & (current_candle == 'red'):
                showlegend = True
                shown_red = True
            else:
                showlegend = False

            fig.add_trace(
                go.Candlestick(
                    x = [row['Date']],
                    open = [row['Open']],
                    high = [row['High']],
                    low = [row['Low']],
                    close = [row['Close']],
                    name = name,
                    increasing = color_dict,
                    decreasing = color_dict,
                    showlegend = showlegend,
                    legendgroup = 'upper'
                )
            )
        
    else:  # candle_type == 'hollow'
        
        title = f'{tk} Prices - Hollow Candles'
        
        df['Prev Close'] = df['Close'].shift(1)
        
        # Define color based on close and previous close
        df['color'] = np.where(df['Close'] > df['Prev Close'], green_color, red_color)
        
        # Set fill to transparent if close > open and to the previously defined color otherwise

        df['fill'] = np.where(
            df['color'] == green_color,
            np.where(df['Close'] > df['Open'], green_fill_color_hollow, green_color),
            np.where(df['Close'] > df['Open'], red_fill_color_hollow, red_color)
        )
        
        shown_red_fill = False
        shown_red_hollow = False
        shown_green_fill = False
        shown_green_hollow = False
        
        for _, row in df.iterrows():
            
            if (row['color'] == green_color) & (row['fill'] == green_color):
                name = 'Open > Close > Prev Close'
                current_candle = 'green_fill'
            elif (row['color'] == green_color) & (row['fill'] == green_fill_color_hollow):
                name = 'Prev Close < Close > Open'
                current_candle = 'green_hollow'
            elif (row['color'] == red_color) & (row['fill'] == red_color):
                name = 'Open > Close < Prev Close'
                current_candle = 'red_fill'
            elif (row['color'] == red_color) & (row['fill'] == red_fill_color_hollow):
                name = 'Prev Close > Close > Open'
                current_candle = 'red_hollow'
            else:
                name = 'Hollow Candles'
            
            # Make sure each candle type appears only once in the legend
            if (not shown_green_fill) & (current_candle == 'green_fill'):
                showlegend = True
                shown_green_fill = True
            elif (not shown_green_hollow) & (current_candle == 'green_hollow'):
                showlegend = True
                shown_green_hollow = True
            elif (not shown_red_fill) & (current_candle == 'red_fill'):
                showlegend = True
                shown_red_fill = True
            elif (not shown_red_hollow) & (current_candle == 'red_hollow'):
                showlegend = True
                shown_red_hollow = True
            else:
                showlegend = False
        
            color_dict = dict(
                fillcolor = row['fill'],
                line=dict(color = row['color'])
            )
            
            fig.add_trace(
                go.Candlestick(
                    x = [row['Date']],
                    open = [row['Open']],
                    high = [row['High']],
                    low = [row['Low']],
                    close = [row['Close']],
                    increasing = color_dict,
                    decreasing = color_dict,
                    showlegend = showlegend,
                    legendgroup = 'upper',
                    name = name
                )
            )
        
    # Add plot border
    fig.add_shape(
        type = 'rect',
        xref = 'x',  # use 'x' to avoid double lines at x_min and x_max
        yref = 'paper',
        x0 = x_min,
        x1 = x_max,
        y0 = 0,
        y1 = 1,
        line_color = style['x_linecolor'],
        line_width = 0.3
    )
    # Update layout and axes
    fig.update_layout(
        width = plot_width,
        height = plot_height,
        xaxis_rangeslider_visible = False,
        template = style['template'],
        legend_groupclick = 'toggleitem',
        yaxis_title = f'Price',
        margin_t = 60,
        title = dict(
            text = title,
            font_size = title_font_size,
            y = 0.975,
            x = 0.45,
            xanchor = 'center',
            yanchor = 'top'
        )
    )
    fig.update_xaxes(
        type = 'category',
        gridcolor = style['x_gridcolor'],        
        nticks = n_ticks_max,
        tickangle = -90,
        ticks = 'outside',
        ticklen = 8
    )
    fig.update_yaxes(
        range = (y_min, y_max),
        gridcolor = style['y_gridcolor'],
        nticks = n_yticks_max,
        ticks = 'outside',
        ticklen = 8
    )

    fig_data = {
        'fig': fig,
        'y_min': y_min,
        'y_max': y_max
    }

    return fig_data

In [236]:
# Traditional candle patterns:
# https://www.incrediblecharts.com/candlestick_patterns/candlestick-patterns.php

x_min = datetime(2024, 6, 21)
x_max = datetime(2024, 9, 19)

theme = 'dark'
theme = 'light'
color_theme = 'gold'

candle_data = plot_candlestick_plotly(df_ohlc[x_min: x_max], tk, candle_type = 'hollow', theme = theme)
# candle_data = plot_candlestick_plotly(df_ohlc[x_min: x_max], tk, candle_type = 'traditional', theme = theme)

candle_fig = candle_data['fig']

candle_fig.show()

In [33]:
x_min = datetime(2024, 6, 21)
x_max = datetime(2024, 9, 19)

theme = 'dark'
color_theme = 'sapphire'

# candle_data = plot_candlestick_plotly(df_ohlc[x_min: x_max], tk, candle_type = 'traditional', theme = theme)
candle_data = plot_candlestick_plotly(df_ohlc[x_min: x_max], tk, candle_type = 'hollow', theme = theme)
# candle_data = add_price_overlays(candle_data, price_list, x_min, x_max, theme = theme, color_theme = color_theme)

candle_fig = candle_data['fig']
candle_fig.show()

In [36]:
theme = 'light'
theme = 'dark'
color_theme = 'gold'
color_theme = 'gold'

bollinger_list = bollinger_bands(df_adj_close[tk], 20, 2, 3)
fig_hist_data = plot_hist_plotly(df_adj_close, tk, theme = theme)
fig_boll_new = add_bollinger_overlays(fig_hist_data, bollinger_list, theme = theme, color_theme = color_theme)

fig_boll_new['fig'].show()

4


IndexError: invalid index to scalar variable.

In [31]:
theme = 'light'
theme = 'dark'
# color_theme = 'rainbow'

fig_hist_data = plot_hist_plotly(df_adj_close, tk, theme = theme)

# ma_type = 'sma'
window = 50
prc_offset = 2.5
n_bands = 3
color_theme = 'gold'

ma_envelope_list = ma_envelopes(df_adj_close[tk], window = window, prc_offset = prc_offset, n_bands = n_bands)
fig_ma_env = add_ma_envelope_overlays(fig_hist_data, ma_envelope_list, theme = theme, color_theme = color_theme)
fig_ma_env['fig'].show()

window = 10
prc_offset = 2.5
n_bands = 3
color_theme = 'magenta'

ma_envelope_list = ma_envelopes(df_adj_close[tk], window = window, prc_offset = prc_offset, n_bands = n_bands)
fig_ma_env = add_ma_envelope_overlays(fig_hist_data, ma_envelope_list, theme = theme, color_theme = color_theme)
fig_ma_env['fig'].show()

# names = [x['name'] for x in fig_ma_env['fig']['data']]
# print(names)

# bollinger_list = bollinger_bands(df_adj_close[tk], 20, 2, 2)
# fig_boll_new = add_bollinger_overlays(fig_ma_env, bollinger_list, theme = theme, color_theme = 'turquoise')

# fig_boll_new['fig'].show()

In [32]:
from analyze_prices import AnalyzePrices
from mapping_plot_attributes import theme_style

analyze_prices = AnalyzePrices(end_date, start_date, [tk])
theme = 'dark'
color_theme = 'turquoise'
style= theme_style[theme]

# candle_data = plot_candlestick_plotly(df_ohlc, tk, candle_type = 'traditional', theme = 'dark')
# candle_data = analyze_prices.plot_candlestick_plotly(df_ohlc, tk, candle_type = 'traditional', theme = 'dark')
candle_data = plot_candlestick_plotly(df_ohlc, tk, candle_type = 'traditional', theme = 'dark')
candle_fig = candle_data['fig']

bollinger_list = bollinger_bands(df_close[tk], 20, 2, 1)
# candle_data = analyze_prices.add_bollinger_overlays(candle_data, bollinger_list, theme = theme, color_theme = 'sapphire')
candle_data = add_bollinger_overlays(candle_data, bollinger_list, theme = theme, color_theme = 'sapphire')

fig_boll = candle_data['fig']

# fig_boll.show()

window = 20
prc_offset = 10
n_bands = 1

# ma_envelope_list = ma_envelopes(df_adj_close[tk], ma_type, window, prc_offset = prc_offset, n_bands = n_bands)
ma_envelope_list = ma_envelopes(df_close[tk], window = window, prc_offset = prc_offset, n_bands = n_bands)
candle_data = add_ma_envelope_overlays(candle_data, ma_envelope_list, theme = theme, color_theme = 'turquoise')
fig_ma_env = candle_data['fig']
fig_ma_env.show()

2


Moving Average Overlays

In [33]:
ma_list = [
    {
        'ma_idx': 1,
        'ma_type': 'sma',
        # 'ma_window': 10,
        'ma_window': 5,
        'showlegend': True
    },
    {
        'ma_idx': 2,
        'ma_type': 'sma',
        # 'ma_window': 20,
        'ma_window': 10,
        'showlegend': True
    },
    {
        'ma_idx': 3,
        'ma_type': 'sma',
        # 'ma_window': 30,
        'ma_window': 15,
        'showlegend': True
    },
    {
        'ma_idx': 4,
        'ma_type': 'sma',
        # 'ma_window': 40,
        'ma_window': 20,
        'showlegend': True
    }
    ,
    {
        'ma_idx': 5,
        'ma_type': 'sma',
        #'ma_window': 50,
        'ma_window': 25,
        'showlegend': True
    },
    {
        'ma_idx': 6,
        'ma_type': 'sma',
        # 'ma_window': 60,
        'ma_window': 30,
        'showlegend': True
    }
]


In [40]:
theme = 'light'
# theme = 'dark'

color_theme = 'lavender'

from mapping_plot_attributes import theme_style
from analyze_prices import AnalyzePrices
analyze_prices = AnalyzePrices(end_date, start_date, [tk])

x_min = datetime(2023, 9, 20)
x_max = datetime(2024, 9, 19)

# candle_data = plot_candlestick_plotly(df_ohlc[x_min: x_max], tk, candle_type = 'traditional', theme = theme)
candle_data = plot_candlestick_plotly(df_ohlc, tk, candle_type = 'traditional', theme = theme)
# candle_data = analyze_prices.plot_candlestick_plotly(df_ohlc, tk, candle_type = 'traditional', theme = theme)
candle_data = add_ma_overlays(candle_data, close_tk[x_min: x_max], ma_list, theme = theme, color_theme = color_theme)
# candle_data = add_ma_overlays(candle_data, close_tk, ma_list, theme = theme, color_theme = color_theme)

candle_fig = update_color_theme(
    candle_data['fig'],
    candle_data['color_map'],
    theme = theme,
    color_theme = color_theme,
    invert = True
)
candle_fig.show()

IndexError: invalid index to scalar variable.

In [96]:
from mapping_plot_attributes import theme_style
from analyze_prices import AnalyzePrices
analyze_prices = AnalyzePrices(end_date, start_date, [tk])

theme = 'light'
# theme = 'dark'
color_theme = 'magenta'

fig_hist_data = plot_hist_plotly(df_adj_close, tk, theme = theme)
fig_hist = fig_hist_data['fig']

new_data = add_ma_overlays(fig_hist_data, df_adj_close[tk], ma_list, theme = theme, color_theme = color_theme)
new_data_fig = new_data['fig']
new_data_fig.show()

IndexError: invalid index to scalar variable.

In [95]:
print(new_data_fig['color_map'])

NameError: name 'new_data_fig' is not defined

In [38]:
print(new_data['color_map'])

{'SMA 5': 5, 'SMA 10': 4, 'SMA 15': 3, 'SMA 20': 2, 'SMA 25': 1, 'SMA 30': 0}


In [94]:
# https://plotly.com/python/creating-and-updating-figures/

from mapping_plot_attributes import theme_style

theme = 'light'
# theme = 'dark'
style= theme_style[theme]
new_color_theme = 'tableau'

new_data_fig = update_color_theme(new_data_fig, new_data['color_map'], theme, new_color_theme, invert = True)
# new_data_fig = update_color_theme(new_data_fig, new_data['color_map'], theme, new_color_theme)
print(new_data['color_map'])
new_data_fig.show()

NameError: name 'new_data_fig' is not defined